### Перед началом решения выполните следующую ячейку

In [1]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-02-13 18:07:36--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-02-13 18:07:36--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: 'auto_purchase.log'

     0K .......... .......... .......... .......... .......... 11% 3.25M 0s
    50K .......... .......... 

### Задача 1

In [83]:
from collections import namedtuple
from datetime import datetime

log_item = namedtuple('log_item', ['level', 'date', 'file', 'line', 'msg'])

with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()



def count_success_and_failure(file_path):
    logs=[]
    for l in lines:
        splited = l.split('|')
        item = log_item(splited[0].strip(), datetime.strptime(splited[1].strip(), "%Y-%m-%d %H:%M:%S,%f"), splited[2].strip()[len('file: '):], int(splited[3].strip()[len('line: '):]), splited[4].strip()[len('[demon] '):])
        logs.append(item)

    errors = 0
    upd = 0
    for l in logs:
        if l.level == 'ERROR':            
            errors += 1
        elif l.msg.startswith('Обновляем подписку пользователю'):
            upd += 1 

    return (upd - errors, errors)

**Проверка**

In [84]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### Задача 2

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`